In [1]:
import numpy as np
import pandas as pd

In [6]:
# 사용자-평점 행렬 R을 선언한다.
R = np.array([[4, np.nan, np.nan, 2, np.nan],
              [np.nan, 5, np.nan, 3, 1],
              [np.nan, np.nan, 3, 4, 4],
              [5, 2, 1, 2, np.nan]])

# 사용자의 수, 아이템의 수를 R.shape(4, 5)를 이용해 정의
num_users, num_items = R.shape
K = 3 # 3개의 잠재 요인을 설정함

np.random.seed(1)
# 사용자~잠재요인 행렬(P), 아이템~잠재요인 행렬(Q). scale : std.dev of normal dist. 여기서는 K로 나눴다는 거.
# 각 사용자/아이템에게 있어 잠재 요인의 합이 1에 가까워지도록 scaling했다고 보임.
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [7]:
# 실제 R행렬과, 예측 R행렬의 오차를 구하는 get_rmse() 함수를 만들겠습니다.
# get_rmse() 함수는, 실제 R행렬에서 NULL 아닌 행렬 값의 위치 인덱스 추출 --> 실제 R 행렬값과, 예측행렬 값의 RMSE 값을 반환.
# 실제 R행렬에 있는, 실제 값만 가지고 계속 step을 밟아 가는 것!
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0 # initializing.
    full_pred_matrix = np.dot(P, Q.T)

    # 실제 R행렬에서 null 아닌 값의 위치 인덱스를 추출, 실제 R행렬과 예측 행렬의 RMSE 계산
    # non_zeros : 행렬의 모든 index 중, non_zero인 index의 (x, y)쌍을 모아 놓음
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]

    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind] # indexing : 각각 쌍을 주면, 그에 해당하는 값을 1차원 array로 반환
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]

    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

In [8]:
# 행렬 분해 수행 - p, q의 변화율을 계산해 새로운 p, q를 계산하기.
# 실제 R 행렬에서 null이 아닌 요소들의 정보를 가져옴. index, 그 index의 R행렬값
non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j]>0]

steps = 1000
learning_rate = 0.01
r_lambda = 0.01

# SGD 이용, p, q를 업데이트
for step in range(steps):
    for i, j, r in non_zeros:
        # 이 index에서, 실제값과 예측값의 차이 구하기
        eij = r - np.dot(P[i, :], Q[j, :].T) # 사용자 i의 잠재요인, 아이템 j의 잠재요인
        # SGD 업데이트
        P[i, :] = P[i, :] + learning_rate*(eij * Q[j, :] - r_lambda * P[i, :])
        Q[j, :] = Q[j, :] + learning_rate*(eij * P[i, :] - r_lambda * Q[j, :])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step%50)==0:
        print("### iteration step : ", step, "rmse :", rmse)

### iteration step :  0 rmse : 3.2388050277987723
### iteration step :  50 rmse : 0.4876723101369648
### iteration step :  100 rmse : 0.1564340384819247
### iteration step :  150 rmse : 0.07455141311978046
### iteration step :  200 rmse : 0.04325226798579314
### iteration step :  250 rmse : 0.029248328780878973
### iteration step :  300 rmse : 0.022621116143829466
### iteration step :  350 rmse : 0.019493636196525135
### iteration step :  400 rmse : 0.018022719092132704
### iteration step :  450 rmse : 0.01731968595344266
### iteration step :  500 rmse : 0.016973657887570753
### iteration step :  550 rmse : 0.016796804595895633
### iteration step :  600 rmse : 0.01670132290188466
### iteration step :  650 rmse : 0.01664473691247669
### iteration step :  700 rmse : 0.016605910068210026
### iteration step :  750 rmse : 0.016574200475705
### iteration step :  800 rmse : 0.01654431582921597
### iteration step :  850 rmse : 0.01651375177473524
### iteration step :  900 rmse : 0.016481465738

In [11]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬 : \n', np.round(pred_matrix, 3), sep='')

예측 행렬 : 
[[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]


In [13]:
print('실제 행렬 : \n', R, sep='') # 미쳤다 오졌다그냥

실제 행렬 : 
[[ 4. nan nan  2. nan]
 [nan  5. nan  3.  1.]
 [nan nan  3.  4.  4.]
 [ 5.  2.  1.  2. nan]]
